she said to do article and author nodes
    she said authors btwn authors aren't needed 

In [10]:
import pandas as pd
from neo4j import GraphDatabase

In [11]:
uri = "neo4j://localhost:7687"
conn = GraphDatabase.driver(uri, auth=("neo4j", "Ass2DB"), max_connection_lifetime=1000)

## Article Nodes

In [3]:
art = pd.read_csv('Citation Dataset\\ArticleNodes.csv', header=None)
art.columns = ['id','title','year','journal','abstract']
art.head() # 29555

,id,title,year,journal,abstract
0,1001,Compactification Geometry and Duality: N=2,2000,NaN,These are notes based on lectures given at T...
1,1002,Domain Walls and Massive Gauged Supergravity P...,2000,Class.Quant.Grav.,We point out that massive gauged supergravit...
2,1003,Comment on Metric Fluctuations in Brane Worlds,2000,NaN,Recently- Ivanov and Volovich (hep-th/991224...
3,1004,Moving Mirrors and Thermodynamic Paradoxes,2000,Phys.Rev.,Quantum fields responding to moving mirrors...
4,1005,Bundles of chiral blocks and boundary conditio...,2000,NaN,Various aspects of spaces of chiral blocks a...


### Author Attributes

In [4]:
aut = pd.read_csv('Citation Dataset\\AuthorNodes.csv', header=None)
aut.columns = ['id','author']
#aut = aut.groupby('id')['author'].apply(list).reset_index(name='authors') # 29554
aut.head() 

,id,author
0,1001,Paul S. Aspinwall
1,1002,C.N. Pope
2,1002,M. Cvetic
3,1002,H. Lu
4,1003,Y.S. Myung


### Nodes
https://pythontic.com/database/neo4j/create%20nodes%20and%20relationships

In [40]:
nodes = aut.merge(art, how='left', on='id')
nodes

,id,author,title,year,journal,abstract
0,1001,Paul S. Aspinwall,Compactification Geometry and Duality: N=2,2000,NaN,These are notes based on lectures given at T...
1,1002,C.N. Pope,Domain Walls and Massive Gauged Supergravity P...,2000,Class.Quant.Grav.,We point out that massive gauged supergravit...
2,1002,M. Cvetic,Domain Walls and Massive Gauged Supergravity P...,2000,Class.Quant.Grav.,We point out that massive gauged supergravit...
3,1002,H. Lu,Domain Walls and Massive Gauged Supergravity P...,2000,Class.Quant.Grav.,We point out that massive gauged supergravit...
4,1003,Y.S. Myung,Comment on Metric Fluctuations in Brane Worlds,2000,NaN,Recently- Ivanov and Volovich (hep-th/991224...
...,...,...,...,...,...,...
58335,9912292,E. Ivanov,1/4 PBGS and Superparticle Actions,1999,NaN,We construct the worldline superfield massiv...
58336,9912292,S. Krivonos,1/4 PBGS and Superparticle Actions,1999,NaN,We construct the worldline superfield massiv...
58337,9912293,L. Cornalba,Corrections to the Abelian Born-Infeld Action ...,1999,JHEP,In a recent paper Seiberg and Witten have ar...
58338,9912294,A. Y. Marko,Relativistic particle interaction with a weak ...,1999,NaN,Schrodinger equation with two-component wave...


In [42]:
nodes[nodes['id'] == '9304045']

C:\Users\Clayton Preston\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,id,author,title,year,journal,abstract


### Create Article Nodes

In [6]:
for index, row in nodes.iterrows():
    article =  'ID' + str(row['id'])
    author = str(row['author']).replace('"','').replace("'",'').replace('`','').replace('{','').replace('}','').replace('\\','').replace('/','')
    title = str(row['title']).replace('"','').replace("'",'').replace('`','').replace('{','').replace('}','').replace('\\','').replace('/','')
    abst = str(row['abstract']).replace('"','').replace("'",'').replace('`','').replace('{','').replace('}','').replace('\\','').replace('/','')
    
    cqlCreate = "CREATE ({}:article".format(article) + " {" + " id: "'"{}"'", author: "'"{}"'", title: "'"{}"'", year: "'"{}"'", journal: "'"{}"'", abstract: "'"{}"'"".format(article, author, title, row['year']
                    , str(row['journal']) , abst ) + "})"    # cont of .formt( then close out

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)

# Create Relationships

## Citation Relationships

In [43]:
cit = pd.read_csv('Citation Dataset\\Citations.csv', sep = '\t', header=None)
cit.columns = ['id1','id2']
cit

,id1,id2
0,1001,9304045
1,1001,9308122
2,1001,9309097
3,1001,9311042
4,1001,9401139
...,...,...
352802,9912286,9805150
352803,9912286,9806074
352804,9912286,9808140
352805,9912286,9810068


In [8]:
for index, row in cit.iterrows():
    id1 =  'ID' + str(row['id1'])
    id2 =  'ID' + str(row['id2'])

    cqlCreate = "CREATE ({})-[:cites]->({})".format(id1,id2)

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(cqlCreate)

StopIteration: 

In [ ]:
for index, row in cit.iterrows():
    id1 =  'ID' + str(row['id1'])
    id2 =  'ID' + str(row['id2'])

    CQLrelate = """
    MATCH
      ({}:article),
      ({}:article)
    WHERE {}.id = '{}' AND {}.name = '{}'
    CREATE ({})-[r:cites]->({})
    RETURN type(r)

    """.format(  id1, id2,  id1, id1,    id2, id2,    id1,id2  )

    with conn.session() as graphDB_Session:
        # Create nodes
        graphDB_Session.run(CQLrelate)

In [20]:
# old way

CQLrelate = """
CREATE (ID1001)-[:cites]->(ID9304045)
"""

with conn.session() as graphDB_Session:
    # Create nodes
    graphDB_Session.run(CQLrelate)

In [36]:
CQLrelate = """
MATCH
  (ID1001:article),
  (ID9304045:article)
WHERE ID1001.id = 'ID1001' AND ID9304045.id = 'ID9304045'
CREATE (ID1001)-[r:cites]->(ID9304045)
RETURN type(r)

"""

with conn.session() as graphDB_Session:
    # Create nodes
    graphDB_Session.run(CQLrelate)

In [29]:
q1 = """

MATCH (a:article {id: 'ID1001'})-[r]-(b)
RETURN r, a, b

"""

In [ ]:
q2 = """

MATCH (x:article {id: 'ID1002'}) RETURN x

"""

In [34]:
q3 = """

MATCH (n1:ID1001)-[:HAS_RELATIONSHIP]-(OtherNodes)
RETURN n1, OtherNodes

"""

In [35]:
# Query the graph    
with conn.session() as graphDB_Session:
    nodes = graphDB_Session.run(q3)

    for node in nodes:

        print(node)